In [24]:
import pybaseball
import polars as pl
import boto3
import s3fs
import datetime as dt
import pandas as pd
import api_handler

In [2]:
start_dates = pl.date_range(start=dt.date(2024, 2, 15),
                            end=dt.date(2024, 11, 30), 
                            interval='1w',
                            eager=True)

In [ ]:
dates = pl.DataFrame({'start_date': pl.date_range(dt.date(2024, 2, 15),
                                                  dt.date(2024, 11, 30),
                                                  interval='1w',
                                                  eager=True)})
dates = dates.with_columns(
    pl.col('start_date').dt.offset_by('1w').alias('end_date')
)

# I know iter_rows is unwise. But this is only about 40 entries.
frames_list = [pybaseball.statcast(start_dt=str(i[0]),
                                   end_dt=str(i[1]),
                                   verbose=False) for i in dates.iter_rows()]
frames_list = [i for i in frames_list if i.shape[0] != 0]

In [6]:
df_big = pl.from_pandas(pd.concat(frames_list))

In [9]:
unique_months = df_big['game_date'].dt.strftime('%B').unique()
month_dict = {k: df_big.filter(
    pl.col('game_date').dt.strftime('%B') == k) for k in unique_months}

In [16]:
fs = s3fs.S3FileSystem()
for key in month_dict:
    if key == 'September':
        print('Skipping over September')
        continue
    df_current = month_dict[key]
    destination = f's3://gthomakos14-statcast/2024/{key}.parquet'
    with fs.open(destination, mode='wb') as output:
        df_current.write_parquet(output)
    print(f'Done with {key}')

/tmp/ipykernel_9422/2235613981.py:9: UserWarning: Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.
  df_current.write_parquet(output)


Done with August
Done with July
Done with March
Skipping over September
Done with May
Done with April
Done with June


In [25]:
type(df_big)

polars.dataframe.frame.DataFrame